# Hands-on 3  - Q-Learning 

Para este hands-on vamos mudar um pouco. No hands-on passado fizemos toda a modelagem para o ambiente, desde os estados até as probabilidades de transição. Como o foco deste hands-on é o Q-learning, vamos usar um framework para lidar com toda a parte do ambiente e seus elementos.

Gym é um conjunto de ferramentos e framework para lidar com problemas de aprendizagem por reforço. Ele foi criado devido a necessidade de padronização nos testes de algoritmos e de benchmarks. Assim, esse framework lida com toda a parte da relação agente-ambiente-ações-recompensas e o desenvolvedor fica livre para focar apenas no algoritmo de RL (Reinforcement Learning) que irá usar. Para mais informações sobre o gym: [gym website](https://gym.openai.com/)

Para começar, vamos importar o pacote do gym:

In [4]:
import gym

O gym possui vários ambientes para testes de algoritmos de aprendizado por reforço. Alguns desses ambientes são bem simples, enquanto que outros bem mais complexos. Para no hands-on vamos usar o ambiente "Taxi-v2". Para conhecer os demais ambientes veja em [gym ambientes](https://gym.openai.com/envs/#classic_control)

In [5]:
env = gym.make('Taxi-v2')

Este ambiente consiste num grid com quatro posições padrões: R, G Y, B. Sempre que o ambiente é inicializado, uma dessas posições recebe a cor azul e indica onde está um passageiro. Outra posição recebe a cor magenta e indica o destino que se deve levar o passageiro. O táxi é representado pela cor amarela quando está vázio e pela cor verde quando está com um passageiro. Abaixo um exemplo de como é o grid:

   ![alt text](rlimages/taxi01.png)
   
no grid há posições onde o taxí pode ir, e as ações que ele pode escolher são: Norte, Sul , Leste, Oeste, Pickup (pegar o passageiro) e Dropoff (liberar o passageiro). Cada movimento que o táxi realiza ele é penalizado em -1. Caso ele pegue o passageiro no canto certo e entregue no local certo ele recebe uma recompensa de +20. Porém, caso o táxi faça um pickup ou dropoff fora do local certo ele é penalizado em -10. Vamos rodar um simulação para mostrar a interação do agente com ambiente com escolhas de ações aleatórias:

In [3]:
for i_episode in range(1):

    observation = env.reset()
    for t in range(100):
        env.render() # renderizar o ambiente grafico do ambiente
        action = env.action_space.sample() # funcao do gym para escolha de acao aleatoria dado o ambiente
        observation, reward, done, info = env.step(action) # aplicacao do acao e retorno o proximo estado, recompensa e demais infos

        if done:
            print("Episode finished after {} timestep".format(t+1))
            break

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B

note que certas vezes o táxi realiza um pickup ou dropoff fora do local indicado. E que a simulação acaba não por ter conseguido o objetivo, mas por causa do limite do número máximo de iterações permitada caso o agente não consiga realizar o objetivo. 

Vale o destaque para as funções **env.step()** e **env.reset()**. A função **env.step()** recebe como argumento a ação tomada e retorna o próximo estado (ou observação), a recompensa, um variável *done* do tipo *bool* que indica se o episódio chegou ao final e uma outra variável denominada info, que para nosso hands-on não será utilizada. a função **env.reset()** deve ser sempre utilizada após a definição do ambiente e **retorna o estado inicial do ambiente** 

***
Para aplicar o Q-Learning nesse framework a primeira coisa que necessitamos e saber a quantidade de estados que esse ambiente possui e a quantidade de ações. Para isso, basta fazer:

In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(6)
Discrete(500)


Nota-se que há 6 possíveis ações que podem ser tomadas nesse ambiente e que ele possui 500 estados. Nota-se também que o tipo dessas variáveis é *Discrete* que indica que os valores são inteiros (o que é bom, pois não teremos problemas com indexação na matrix Q necessária no Q-learning). 

Conhecendo esses valores podemos agora aplicar o Q-Learning no processo de aprendizagem

In [7]:
import numpy as np

Q = np.zeros([env.observation_space.n, env.action_space.n]) # Matrix Q necessaria no Q-Learning

G = 0 # variavel para acumular as recompensas

alpha = 0.618 # learning rate 

for episode in range(1, 10001):  #1000 episodios de aprendizagem

    done = False
    G, reward = 0, 0
    state = env.reset()

    while done != True:

        action = np.argmax(Q[state])
        state2, reward, done, info = env.step(action)
        Q[state, action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) # Q-learning
        G += reward
        state = state2

        if episode % 50 == 0:
            print('Episode {} total reward: {}'.format(episode, G))

Episode 50 total reward: -1
Episode 50 total reward: -2
Episode 50 total reward: -3
Episode 50 total reward: -4
Episode 50 total reward: -5
Episode 50 total reward: -6
Episode 50 total reward: -7
Episode 50 total reward: -8
Episode 50 total reward: -9
Episode 50 total reward: -10
Episode 50 total reward: -11
Episode 50 total reward: -12
Episode 50 total reward: -13
Episode 50 total reward: -14
Episode 50 total reward: -15
Episode 50 total reward: -16
Episode 50 total reward: -17
Episode 50 total reward: -18
Episode 50 total reward: -19
Episode 50 total reward: -20
Episode 50 total reward: -21
Episode 50 total reward: -22
Episode 50 total reward: -23
Episode 50 total reward: -24
Episode 50 total reward: -25
Episode 50 total reward: -26
Episode 50 total reward: -27
Episode 50 total reward: -28
Episode 50 total reward: -29
Episode 50 total reward: -30
Episode 50 total reward: -31
Episode 50 total reward: -32
Episode 50 total reward: -33
Episode 50 total reward: -34
Episode 50 total reward

Episode 2850 total reward: -6
Episode 2850 total reward: -7
Episode 2850 total reward: -8
Episode 2850 total reward: -9
Episode 2850 total reward: -10
Episode 2850 total reward: -11
Episode 2850 total reward: 9
Episode 2900 total reward: -1
Episode 2900 total reward: -2
Episode 2900 total reward: -3
Episode 2900 total reward: -4
Episode 2900 total reward: -5
Episode 2900 total reward: -6
Episode 2900 total reward: -7
Episode 2900 total reward: 13
Episode 2950 total reward: -1
Episode 2950 total reward: -2
Episode 2950 total reward: -3
Episode 2950 total reward: -4
Episode 2950 total reward: -5
Episode 2950 total reward: -6
Episode 2950 total reward: -7
Episode 2950 total reward: -8
Episode 2950 total reward: -9
Episode 2950 total reward: -10
Episode 2950 total reward: -11
Episode 2950 total reward: -12
Episode 2950 total reward: -13
Episode 2950 total reward: -14
Episode 2950 total reward: -15
Episode 2950 total reward: -16
Episode 2950 total reward: -17
Episode 2950 total reward: -18


Episode 5050 total reward: -13
Episode 5050 total reward: 7
Episode 5100 total reward: -1
Episode 5100 total reward: -2
Episode 5100 total reward: -3
Episode 5100 total reward: -4
Episode 5100 total reward: -5
Episode 5100 total reward: -6
Episode 5100 total reward: -7
Episode 5100 total reward: -8
Episode 5100 total reward: -9
Episode 5100 total reward: -10
Episode 5100 total reward: -11
Episode 5100 total reward: -12
Episode 5100 total reward: -13
Episode 5100 total reward: 7
Episode 5150 total reward: -1
Episode 5150 total reward: -2
Episode 5150 total reward: -3
Episode 5150 total reward: -4
Episode 5150 total reward: -5
Episode 5150 total reward: -6
Episode 5150 total reward: -7
Episode 5150 total reward: -8
Episode 5150 total reward: -9
Episode 5150 total reward: -10
Episode 5150 total reward: 10
Episode 5200 total reward: -1
Episode 5200 total reward: -2
Episode 5200 total reward: -3
Episode 5200 total reward: -4
Episode 5200 total reward: -5
Episode 5200 total reward: -6
Episod

**Espere até o código acima termine de rodar (aparecerá diversas informações logo abaixo dele assim que ele terminar)**

Com o código acima executado, nosso agente (o táxi) já aprendeu a dinâmica do ambiente e já está pronto para executar a melhor ação dado cada estado. Para testar, vamos rodar o seguinte código dado a matrix Q já definida:

In [6]:
state = env.reset()
done = False
G = 0
env.render()
while done != True:

    action = np.argmax(Q[state])
    state2, reward, done, info = env.step(action)
    env.render()
    G += reward
    state = state2

print('Accumulative Reward: {}'.format(G))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : :_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| :_: : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+-

Agora, notamos que o agente consegue executar o objetivo dado a matrix Q e estado inicial. Análise a saída do código anterior e certifique-se que de fato o agente está fazendo o objetivo.

****

Agora, vamos colocar a mão na massa um pouco. Para isso, vamos utilizar um ambiente diferente agora. Vamos usar o ambiente "CartPole-v0". Veja no link como é; esse ambiente e como ele funciona: [CartPole ](https://gym.openai.com/envs/CartPole-v1/)

Vemos que o objetivo nesse ambiente e fazer com que o carro consiga equilibrar o poste. E que para isso ele pode se mover para direita ou esquerda, bem como modificar a velocidade nos dois sentidos. 

Exposto isso, vamos seguir e observar agora quais são os estados e ações desse ambiente

In [9]:
env = gym.make('CartPole-v0')
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


Aqui temos um pequeno problema. Os estados nesse ambiente são do tipo *box(4,)*, e isso indica que cada estado é representado por um vetor com 4 elementos. E esse não é o único problema, se fizermos:

In [10]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.80000000e+00 3.40282347e+38 4.18879020e-01 3.40282347e+38]
[-4.80000000e+00 -3.40282347e+38 -4.18879020e-01 -3.40282347e+38]


notamos que os valores não são inteiros para esses elementos, o que indica que não será possível usar esse array como indexação na matrix Q. Assim, temos que fazer uma discretização para poder utilizar esse ambiente. 

Abaixo é definida a uma função para discretizar as observações (estados):

In [11]:
import math
buckets=(1, 1, 6, 12)
def discretize(obs):

    upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]
    lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
    ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
    new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
    new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
    
    return tuple(new_obs)


Esta função será utilizada sempre que tivermo um estado retornado do ambiente e antes de utilizá-lo para o cálculo do valor Q na matrix.

Assim, o que se pede é: Utilizando como base o código no exemplo do Táxi. Faça com que o agente consiga o objetivo nesse ambiente do "CartPole".

Para auxiliar, abaixo se encontra um esbouço do código. Complete com o que falta

In [12]:
Q = np.zeros(buckets + (env.action_space.n,))

G = 0 # variavel para acumular as recompensas

alpha = 0.1 
gamma = 1.0

n_episodios = ?
for episode in range(1, n_episodios+1):  

    done = False
    G, reward = 0, 0
    current_state = discretize(env.reset())

    while done != True:

        action = np.argmax()
        state2, ?, ?, ? = env.step(action)
        state2 = discretize(state2)
        ?
        ?

        if episode % 50 == 0:
            print('Episode {} total reward: {}'.format(episode, G))

SyntaxError: invalid syntax (<ipython-input-12-e469afff6758>, line 8)

In [13]:
Q = np.zeros(buckets + (env.action_space.n,))

G = 0 # variavel para acumular as recompensas

alpha = 0.1 
gamma = 1.0

n_episodios = 1000
for episode in range(1, n_episodios+1):  

    done = False
    G, reward = 0, 0
    current_state = discretize(env.reset())

    while done != True:

        action = np.argmax(Q[current_state])
        state2, reward, done, _ = env.step(action)
        state2 = discretize(state2)
        Q[current_state][action] += alpha * (reward + gamma * np.max(Q[state2]) - Q[current_state][action])
        G += reward
        current_state = state2

        if episode % 50 == 0:
            print('Episode {} total reward: {}'.format(episode, G))

Episode 50 total reward: 1.0
Episode 50 total reward: 2.0
Episode 50 total reward: 3.0
Episode 50 total reward: 4.0
Episode 50 total reward: 5.0
Episode 50 total reward: 6.0
Episode 50 total reward: 7.0
Episode 50 total reward: 8.0
Episode 50 total reward: 9.0
Episode 50 total reward: 10.0
Episode 100 total reward: 1.0
Episode 100 total reward: 2.0
Episode 100 total reward: 3.0
Episode 100 total reward: 4.0
Episode 100 total reward: 5.0
Episode 100 total reward: 6.0
Episode 100 total reward: 7.0
Episode 100 total reward: 8.0
Episode 100 total reward: 9.0
Episode 100 total reward: 10.0
Episode 150 total reward: 1.0
Episode 150 total reward: 2.0
Episode 150 total reward: 3.0
Episode 150 total reward: 4.0
Episode 150 total reward: 5.0
Episode 150 total reward: 6.0
Episode 150 total reward: 7.0
Episode 150 total reward: 8.0
Episode 150 total reward: 9.0
Episode 200 total reward: 1.0
Episode 200 total reward: 2.0
Episode 200 total reward: 3.0
Episode 200 total reward: 4.0
Episode 200 total 

Para testar se o agente de fato aprendeu a dinâmica do ambiente, rode o seguinte código e veja o resultado

In [14]:
done = False
G = 0
env.render()
n = 0
while (done != True):
    current_state = discretize(env.reset())
    action = np.argmax(Q[current_state])
    state2, reward, done, info = env.step(action)
    env.render()
    G += reward
    current_state = state2
    n += 1
    if n == 1000: done = True

env.reset()
env.close()
print('Accumulative Reward: {}'.format(G))

Accumulative Reward: 1000.0
